In [1]:
from pynq import Overlay
from pynq.lib.video import *

overlay = Overlay("/home/xilinx/lastTry/base.bit")

In [2]:
imgen_vdma = overlay.video.axi_vdma_0.readchannel

In [3]:
pixgen = overlay.pixel_generator_0
pixgen.register_map

RegisterMap {
  gp0 = Register(value=0),
  gp1 = Register(value=0),
  gp2 = Register(value=0),
  gp3 = Register(value=0),
  gp4 = Register(value=0),
  gp5 = Register(value=0),
  gp6 = Register(value=0),
  gp7 = Register(value=0)
}

In [4]:
#fixed parameters
w = 1024
h = 768
v_w = 640
v_h = 480
speed = 5
x_bound = 120 
y_bound = 90

In [5]:
def dec_to_hex(number0, integers=3, fractional=28):
    # Total number of bits (1 sign bit + integer bits + fractional bits)
    number = number0
    total_bits = 1 + integers + fractional

    if number < 0:
        is_negative = True
        number = -number
    else:
        is_negative = False

    integer_part = int(number)
    fractional_part = number - integer_part

    # Convert integer part to binary with the specified number of bits
    integer_bin = bin(integer_part)[2:].zfill(integers)

    # Convert fractional part to binary with the specified number of bits
    fractional_bin = ""
    for _ in range(fractional):
        fractional_part *= 2
        bit = int(fractional_part)
        fractional_bin += str(bit)
        fractional_part -= bit

    combined_bin = f"{integer_bin[:integers]}{fractional_bin[:fractional]}"
    
    if is_negative:
        inverted_bin = ''.join('1' if bit == '0' else '0' for bit in combined_bin)
        twos_complement_bin = bin(int(inverted_bin, 2) + 1)[2:].zfill(integers + fractional)
    else:
        twos_complement_bin = combined_bin

    final_bin = f"{'1' if is_negative else '0'}{twos_complement_bin}"


    hex_representation = hex(int(final_bin, 2))[2:].zfill((total_bits + 3) // 4)
    
    # Return an int, from the HEX representation
    return int(hex_representation, 16)

In [6]:
def get_zoom_factor(w, r1, r2):
    #We assume that we always have a 4/3 ratio
    if r2 > r1:
        d = r2-r1
    else:
        d = r1-r2
    z_f = d/w
    return z_f

In [7]:
def to_32bit_hex(w, h):
    if not (0 <= w < 2**16):
        raise ValueError("w must be a 16-bit number (0 <= w < 65536)")
    if not (0 <= h < 2**16):
        raise ValueError("h must be a 16-bit number (0 <= h < 65536)")
    
    combined = (h << 16) | w
    
    return combined

In [8]:
def update_reg_map(colour, zoom, Im, Re, dim, it):
    pixgen.register_map.gp0 =  colour #0x12 #colour
    print(colour)
    pixgen.register_map.gp1 = zoom   #Zoom
    pixgen.register_map.gp2 = Im     #Im_upper
    pixgen.register_map.gp3 = Re     #Re_lower
    pixgen.register_map.gp4 = dim    #dimensions 
    pixgen.register_map.gp5 = it     #max_iterations
    pixgen.register_map.gp6 = 0x047AE147
    pixgen.register_map.gp7 = 0x0020C49B

In [9]:
import cv2
import numpy as np
from pynq.buffer import PynqBuffer
from pynq import allocate

def resize_small_frame(frame: PynqBuffer, new_height: int, new_width: int) -> PynqBuffer:
    # Convert the PynqBuffer to a NumPy array
    frame_np = np.array(frame)
    
    # Resize the array using OpenCV
    resized_np = cv2.resize(frame_np, (new_width, new_height))
    
    # Allocate a new PynqBuffer for the resized array
    resized = allocate(shape=resized_np.shape, dtype=resized_np.dtype)
    
    # Copy the resized array into the new PynqBuffer
    np.copyto(resized, resized_np)
    
    return resized

In [10]:
import numpy as np
from pynq.buffer import PynqBuffer
from pynq import allocate

def crop_big_frame(big_frame: PynqBuffer, x: int, y: int, zoom_x:int = 0, zoom_y:int = 0) -> PynqBuffer:
    # Convert the PynqBuffer to a NumPy array
     
    crop_height = 480
    crop_width = 640
    zoom_x = int(zoom_x //2)
    zoom_y = int(zoom_y //2)
    big_frame_np = np.array(big_frame)
    
    # Ensure the crop area is within the bounds
    max_y, max_x, _ = big_frame_np.shape
    if y + crop_height > max_y:
        y = max_y - crop_height
    if x + crop_width > max_x:
        x = max_x - crop_width
    if y < 0:
        y = 0
    if x < 0:
        x = 0
    
    # Crop the big np.array 
    small_frame_np = big_frame_np[y-zoom_y:y + crop_height+zoom_y, x-zoom_x:x + crop_width+ zoom_x]
    #print(len(small_frame_np),zoom_y,zoom_x)
    # Allocate a new PynqBuffer for the np.array
    small_frame = allocate(shape=small_frame_np.shape, dtype=small_frame_np.dtype)
    
    # Copy the np.array into PynqBuffer
    np.copyto(small_frame, small_frame_np)
    
    # In case of zoom - resize to the correct format for HDMI display
    small_frame = resize_small_frame(small_frame, 480, 640) 
    
    return small_frame

In [11]:
videoMode = common.VideoMode(w,h, 24)
imgen_vdma.mode = videoMode
imgen_vdma.start()

In [12]:
screenMode = common.VideoMode(v_w, v_h, 24)

In [13]:
hdmi_out = overlay.video.hdmi_out
hdmi_out._vdma = overlay.video.axi_vdma #Use the correct VDMA!
hdmi_out.configure(screenMode)
hdmi_out.start()

In [14]:
Re = -2
colour = 2
r2 = 2
zoom = get_zoom_factor(w, Re, r2)
Im = 1.5
dim = to_32bit_hex(w, h)
it = 0x3ff
x = (w-v_w)//2
y = (h-v_h)//2

center_x = (w-v_w)//2
center_y = (h-v_h)//2

image_w = 640
image_h = 480
update_reg_map(colour, dec_to_hex(zoom),  dec_to_hex(Im),  dec_to_hex(Re), dim, it)
loading = False

2


In [15]:
#reset to orignial values
def reset():
    Re = -2
    r2 = 2
    zoom = get_zoom_factor(w, Re, r2)
    Im = 1.5
    dim = to_32bit_hex(w, h)
    it = 0x3ff
    x = (w-v_w)//2
    y = (h-v_h)//2

    center_x = (w-v_w)//2
    center_y = (h-v_h)//2

    image_w = 640
    image_h = 480
    update_reg_map(colour, dec_to_hex(zoom),  dec_to_hex(Im),  dec_to_hex(Re), dim, it)
    loading = True
    return Re, Im, zoom, image_w, image_h
reset()

2


(-2, 1.5, 0.00390625, 640, 480)

In [16]:
frame = imgen_vdma.readframe()

In [17]:
def get_image_parameters(zoom, Re, Im, image_w, image_h):
    # Distances 
    d_x = zoom*w
    d_y = zoom*h
    
    small_d_x = (image_w*d_x)/w
    small_d_y = (image_h*d_y)/h
    
    
    # Proportion parameters
    p_x = (((v_w-image_w)/2)+x)/w
    p_y = (((v_h-image_h)/2)+y)/h
    print("p", p_x, p_y)
    
    
    image_Re = Re + (d_x*p_x)
    image_Im = Im - (d_y*p_y)
    

    
    print("Small_d", small_d_x, small_d_y)
    return image_Re, image_Im, small_d_x , small_d_y

In [18]:
def generate_new_frame(zoom, Re, Im, image_w, image_h):
    image_Re, image_Im, d_x, d_y = get_image_parameters(zoom, Re, Im, image_w, image_h)
    
    big_d_x = (w*d_x)/v_w
    big_d_y = (h*d_y)/v_h
        
    c_x = d_x/2 + min(-image_Re, image_Re)
    c_y =  max(-image_Im, image_Im) - d_y/2 
    print("centre", c_x, c_y)
    
    new_Re = c_x - big_d_x/2
    new_Im = c_y + big_d_y/2
    
    print("big_distance", big_d_x, big_d_y)
    
    new_zoom = get_zoom_factor(w, new_Re, new_Re+big_d_x) 
    update_reg_map(colour, dec_to_hex(new_zoom), dec_to_hex(new_Im), dec_to_hex(new_Re), dim, it)
    
    return new_Re, new_Im, new_zoom, image_Re, image_Im

In [19]:
import time
start_time = None
loading = False
loading_time = 1.2

In [20]:
# movemement
def move_right (frame, x, y):
    x += speed
    out = False
    if image_w+x > w - x_bound:
        out = True
    cropped_frame = crop_big_frame(frame, x, y)
    return cropped_frame, x , y, out

def move_left (frame, x, y):
    x -= speed
    out = False
    if x_bound > x-(image_w-v_w)/2:
        out = True
    cropped_frame = crop_big_frame(frame, x, y)
    return cropped_frame, x , y, out

def move_down (frame, x, y):
    y += speed
    out = False
    if image_h+y > h - y_bound:
        out = True
    cropped_frame = crop_big_frame(frame, x, y)
    return cropped_frame, x ,y, out

def move_up (frame, x, y):
    y -= speed
    out = False
    if y_bound > y - (image_h - v_h)/2:
        out = True
    cropped_frame = crop_big_frame(frame, x, y)
    return cropped_frame, x, y, out

def zoom_out(frame, x, y, image_w, image_h):
    out = False
    image_w += 8
    image_h += 6
    #print(image_w , w-x_bound)
    #print(image_h , h-y_bound)
    if image_w > w-(2*x_bound) or image_h > h-(2*y_bound):
        out = True
    #cropped_frame = zoom_out_pynq_buffer(frame, 1.1)
    z_x = image_w - 640
    z_y = image_h - 480
    cropped_frame = crop_big_frame(frame, x, y, z_x, z_y)
    return cropped_frame, image_w, image_h, out
def zoom_in(frame, x, y, image_w, image_h):
    out = False
    image_w -= 8
    image_h -= 6
    #print(image_w , w-x_bound)
    #print(image_h , h-y_bound)
    if image_w < v_w-x_bound or image_h < v_h-y_bound:
        out = True
    #cropped_frame = zoom_out_pynq_buffer(frame, 1.1)
    z_x = image_w - 640
    z_y = image_h - 480
    cropped_frame = crop_big_frame(frame, x, y, z_x, z_y)
    return cropped_frame, image_w, image_h, out

In [29]:
import socket
host='192.168.2.99'
port=12344

In [50]:
get_zoom_factor(w, -1.2731, 0.03570814-1.2731)

3.487123046875011e-05

In [30]:
presets = {
    0 : (),
    1 : (0.00012051757812500002, -0.84, 0.244448917388916),
    2:  (0.00048828125, -1, 0.25),
    3:  (1, 2, 3),
    5 : (0.0000034871,-1.2731, 0.39503)
}

colour_map = {
    0 : 5,
    1 : 2,
    2 : 0x0FF00007,
    3 : 0,
    4 : 4,
    5 : 4
}

In [33]:
# Create a TCP/IP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the address and port
server_socket.bind((host, port))


# Listen for incoming connections
server_socket.listen(1)
print(f"Server listening on {host}:{port}")
my_preset , my_theme, is_julia = 0, 0, 0
send_message = "ACK"
while True:
    # Wait for a connection
    client_socket, client_address = server_socket.accept()
    try:
        print(f"Connection from {client_address}")

        # Receive the data in small chunks and retransmit it
        loading = False
        start_time = None
        preset = client_socket.recv(1024).decode()
        client_socket.sendall(str(zoom).encode())
        if preset[0] == "P":
            if preset[1:] != my_preset: 
                my_preset = preset[1:]
                zoom , Re, Im = presets[int(my_preset)]
        theme = client_socket.recv(1024).decode()
        client_socket.sendall(str(zoom).encode())
        if theme[0] == "T":
            if theme[1:] != my_theme:
                print("changed to ", theme[1:])
                my_theme = theme[1:]
                colour = colour_map[int(my_theme)]
        julia = client_socket.recv(1024).decode()
        client_socket.sendall(str(zoom).encode())
        
        if julia[0] == "J":
            if julia[1:] != is_julia:
                is_julia = julia[1:]
                
                update_reg_map(colour, dec_to_hex(zoom), dec_to_hex(Im), dec_to_hex(Re), dim, it)
                time.sleep(2)
                frame = imgen_vdma.readframe() 
            if is_julia == "1":
                print("julia")
                my_theme = int(my_theme)
                my_theme += 16
                colour = int(my_theme)
                update_reg_map(colour, dec_to_hex(zoom), dec_to_hex(Im), dec_to_hex(Re), dim, it)
                
        update_reg_map(colour, dec_to_hex(zoom), dec_to_hex(Im), dec_to_hex(Re), dim, it)
        time.sleep(2)
        frame = imgen_vdma.readframe()  
                
        print(preset, theme, julia)
        connection = True
        while connection:
            data = client_socket.recv(1024)
            
            client_socket.sendall(str(zoom).encode())
            print("sent ACK")
            if data:
                message = data.decode('utf-8')
                print(f"Received message:{message}")
                print(get_image_parameters(zoom, Re, Im, image_w, image_h))
                #respond
                if message == "move_left_down":
                    cropped_frame, x, y, out = move_left(frame, x, y)
                    cropped_frame, x, y, out = move_down(frame, x, y)
                if message == "move_left_up":
                    cropped_frame, x, y, out = move_left(frame, x, y)
                    cropped_frame, x, y, out = move_up(frame, x, y)
                if message == "move_right_down":
                    cropped_frame, x, y, out = move_right(frame, x, y)
                    cropped_frame, x, y, out = move_down(frame, x, y)
                if message == "move_right_up":
                    cropped_frame, x, y, out = move_right(frame, x, y)
                    cropped_frame, x, y, out = move_up(frame, x, y)
                if message == "move_left":
                    cropped_frame, x, y, out = move_left(frame, x, y)
                if message == "move_right":
                    cropped_frame, x, y, out = move_right(frame, x, y)
                if message == "move_up":
                    cropped_frame, x, y, out = move_up(frame, x, y)
                if message == "move_down":
                    cropped_frame, x, y, out = move_down(frame, x, y)
                if message == "zoom_in":
                    cropped_frame, image_w, image_h, out= zoom_in(frame, x, y, image_w, image_h)
                if message == "zoom_out":
                    cropped_frame, image_w, image_h, out= zoom_out(frame, x, y, image_w, image_h)
                if message == "reset":
                    Re, Im, zoom, image_w, image_h = reset()
                    time.sleep(loading_time)
                    frame = imgen_vdma.readframe()
                if message == "close":
                    print("Connection closed")
                    client_socket.close()
                    connection = False
                if out and not loading:
                    print("out")
                    new_Re, new_Im, zoom, image_Re, image_Im = generate_new_frame(zoom, Re, Im, image_w, image_h)
                    og_x, og_y = x, y
                    og_w, og_h = image_w, image_h
                    print("Image", image_Re, image_Im)
                    print("Frame", new_Re, new_Im)
                    #new_x, new_y = get_new_xy(x, y, Re, new_Re, Im, new_Im, image_Re, image_Im)
                    #print(new_x, new_y)
                    Re = new_Re
                    Im = new_Im

                    loading = True
                    start_time = time.time()
                    x_buffer = 0
                    y_buffer = 0
                
                elif loading:
                    if time.time() - start_time >= loading_time:
                        bg_frame = imgen_vdma.readframe()
                        frame = bg_frame
                        image_w = 640 + (image_w - og_w)
                        image_h = 480 + (image_h - og_h)
                        loading = False
                        x, y = center_x + (x - og_x), center_y + (y - og_y)
                        print("image loaded")
                    else:
                        print("buffer")
                hdmi_out.writeframe(cropped_frame)
            else:
                break
    finally:
        # Clean up the connection
        client_socket.close()

Server listening on 192.168.2.99:12344
Connection from ('192.168.2.4', 32555)
changed to  0
5
5
P2 T0 J0
sent ACK
Received message:move_up
p 0.205078125 0.25390625
Small_d 0.24609375 0.1845703125
(-0.8974609375, 0.15478515625, 0.24609375, 0.1845703125)
sent ACK
Received message:move_up
p 0.205078125 0.24739583333333334
Small_d 0.24609375 0.1845703125
(-0.8974609375, 0.1572265625, 0.24609375, 0.1845703125)
sent ACK
Received message:move_up
p 0.205078125 0.24088541666666666
Small_d 0.24609375 0.1845703125
(-0.8974609375, 0.15966796875, 0.24609375, 0.1845703125)
sent ACK
Received message:move_up
p 0.205078125 0.234375
Small_d 0.24609375 0.1845703125
(-0.8974609375, 0.162109375, 0.24609375, 0.1845703125)
sent ACK
Received message:move_up
p 0.205078125 0.22786458333333334
Small_d 0.24609375 0.1845703125
(-0.8974609375, 0.16455078125, 0.24609375, 0.1845703125)
sent ACK
Received message:move_up
p 0.205078125 0.22135416666666666
Small_d 0.24609375 0.1845703125
(-0.8974609375, 0.1669921875, 0.2

sent ACK
Received message:move_up
p 0.158203125 0.03125
Small_d 0.24609374999999994 0.1845703125
(-0.9107666015624999, 0.2930908203125, 0.24609374999999994, 0.1845703125)
buffer
sent ACK
Received message:move_up
p 0.158203125 0.024739583333333332
Small_d 0.24609374999999994 0.1845703125
(-0.9107666015624999, 0.295013427734375, 0.24609374999999994, 0.1845703125)
buffer
sent ACK
Received message:move_up
p 0.158203125 0.018229166666666668
Small_d 0.24609374999999994 0.1845703125
(-0.9107666015624999, 0.29693603515625, 0.24609374999999994, 0.1845703125)
buffer
sent ACK
Received message:move_up
p 0.158203125 0.01171875
Small_d 0.24609374999999994 0.1845703125
(-0.9107666015624999, 0.298858642578125, 0.24609374999999994, 0.1845703125)
buffer
sent ACK
Received message:move_up
p 0.158203125 0.005208333333333333
Small_d 0.24609374999999994 0.1845703125
(-0.9107666015624999, 0.30078125, 0.24609374999999994, 0.1845703125)
buffer
sent ACK
Received message:move_up
p 0.158203125 -0.00130208333333333

buffer
sent ACK
Received message:zoom_in
p 0.265625 0.22005208333333334
Small_d 0.14765625 0.11074218749999998
(-0.85770263671875, 0.26886596679687497, 0.14765625, 0.11074218749999998)
buffer
sent ACK
Received message:zoom_in
p 0.26953125 0.22395833333333334
Small_d 0.14519531249999998 0.10889648437499998
(-0.85647216796875, 0.267943115234375, 0.14519531249999998, 0.10889648437499998)
buffer
sent ACK
Received message:zoom_in
p 0.2734375 0.22786458333333334
Small_d 0.14273437499999997 0.10705078124999999
(-0.8552416992187499, 0.267020263671875, 0.14273437499999997, 0.10705078124999999)
buffer
sent ACK
Received message:zoom_in
p 0.27734375 0.23177083333333334
Small_d 0.1402734375 0.10520507812499998
(-0.8540112304687499, 0.266097412109375, 0.1402734375, 0.10520507812499998)
buffer
sent ACK
Received message:zoom_in
p 0.28125 0.23567708333333334
Small_d 0.13781249999999998 0.10335937499999998
(-0.85278076171875, 0.265174560546875, 0.13781249999999998, 0.10335937499999998)
buffer
sent ACK
R

buffer
sent ACK
Received message:zoom_in
p 0.28125 0.28125
Small_d 0.07622753906249997 0.057170654296874984
(-0.8219882812500001, 0.23131359863281248, 0.07622753906249997, 0.057170654296874984)
buffer
sent ACK
Received message:zoom_in
p 0.28515625 0.28515625
Small_d 0.07486633300781248 0.05614974975585935
(-0.8213076782226564, 0.23080314636230465, 0.07486633300781248, 0.05614974975585935)
buffer
sent ACK
Received message:zoom_in
p 0.2890625 0.2890625
Small_d 0.07350512695312497 0.055128845214843736
(-0.8206270751953126, 0.23029269409179684, 0.07350512695312497, 0.055128845214843736)
buffer
sent ACK
Received message:move_down
p 0.29296875 0.29296875
Small_d 0.07214392089843748 0.054107940673828105
(-0.8199464721679689, 0.22978224182128903, 0.07214392089843748, 0.054107940673828105)
buffer
sent ACK
Received message:move_down
p 0.29296875 0.2994791666666667
Small_d 0.07214392089843748 0.054107940673828105
(-0.8199464721679689, 0.22893148803710933, 0.07214392089843748, 0.054107940673828105

buffer
sent ACK
Received message:zoom_in
p 0.302734375 0.390625
Small_d 0.05227031249999996 0.03920273437499997
(-0.8130723815917971, 0.20395335693359373, 0.05227031249999996, 0.03920273437499997)
buffer
sent ACK
Received message:move_left
p 0.306640625 0.39453125
Small_d 0.05118134765624996 0.03838601074218747
(-0.8125278991699221, 0.2035449951171875, 0.05118134765624996, 0.03838601074218747)
image loaded
sent ACK
Received message:move_left_down
p 0.2490234375 0.25390625
Small_d 0.06860478515624994 0.051453588867187455
(-0.8205590148925783, 0.21824602050781247, 0.06860478515624994, 0.051453588867187455)
sent ACK
Received message:move_left_down
p 0.244140625 0.2604166666666667
Small_d 0.06860478515624994 0.051453588867187455
(-0.8212396179199221, 0.21756541748046873, 0.06860478515624994, 0.051453588867187455)
sent ACK
Received message:move_left_down
p 0.2392578125 0.2669270833333333
Small_d 0.06860478515624994 0.051453588867187455
(-0.8219202209472658, 0.21688481445312496, 0.0686047851

...

buffer
sent ACK
Received message:zoom_in
p 0.2373046875 0.26171875
Small_d 6.709984107029811 5.032488080272358
(-9.216306388187935, 7.0963112244708615, 6.709984107029811, 5.032488080272358)
buffer
sent ACK
Received message:zoom_in
p 0.2412109375 0.265625
Small_d 6.599984367570306 4.94998827567773
(-9.161306518458181, 7.055061322173547, 6.599984367570306, 4.94998827567773)
buffer
sent ACK
Received message:zoom_in
p 0.2451171875 0.26953125
Small_d 6.4899846281108005 4.8674884710831
(-9.10630664872843, 7.013811419876233, 6.4899846281108005, 4.8674884710831)
buffer
sent ACK
Received message:zoom_in
p 0.2490234375 0.2734375
Small_d 6.379984888651296 4.784988666488472
(-9.051306778998676, 6.972561517578918, 6.379984888651296, 4.784988666488472)
buffer
sent ACK
Received message:zoom_in
p 0.2529296875 0.27734375
Small_d 6.269985149191791 4.702488861893843
(-8.996306909268924, 6.931311615281604, 6.269985149191791, 4.702488861893843)
buffer
sent ACK
Received message:zoom_in
p 0.2568359375 0.2812

buffer
sent ACK
Received message:zoom_in
p 0.2890625 0.2890625
Small_d 3.3412420860824676 2.5059315645618505
(-7.188186191903308, 5.833032966615607, 3.3412420860824676, 2.5059315645618505)
buffer
sent ACK
Received message:zoom_in
p 0.29296875 0.29296875
Small_d 3.279367232636496 2.459525424477372
(-7.157248765180322, 5.809829896573368, 3.279367232636496, 2.459525424477372)
buffer
sent ACK
Received message:zoom_in
p 0.296875 0.296875
Small_d 3.2174923791905243 2.4131192843928932
(-7.126311338457336, 5.786626826531129, 3.2174923791905243, 2.4131192843928932)
buffer
sent ACK
Received message:zoom_in
p 0.30078125 0.30078125
Small_d 3.1556175257445527 2.3667131443084144
(-7.095373911734351, 5.76342375648889, 3.1556175257445527, 2.3667131443084144)
buffer
sent ACK
Received message:zoom_in
p 0.3046875 0.3046875
Small_d 3.093742672298581 2.320307004223936
(-7.064436485011365, 5.74022068644665, 3.093742672298581, 2.320307004223936)
buffer
sent ACK
Received message:zoom_in
p 0.30859375 0.3085937

buffer
sent ACK
Received message:zoom_in
p 0.296875 0.296875
Small_d 1.750014214369096 1.312510660776822
(-6.392572256046623, 5.236322514723093, 1.750014214369096, 1.312510660776822)
buffer
sent ACK
Received message:zoom_in
p 0.30078125 0.30078125
Small_d 1.716360094861998 1.2872700711464984
(-6.3757451962930745, 5.223702219907931, 1.716360094861998, 1.2872700711464984)
buffer
sent ACK
Received message:zoom_in
p 0.3046875 0.3046875
Small_d 1.6827059753548999 1.262029481516175
(-6.358918136539525, 5.21108192509277, 1.6827059753548999, 1.262029481516175)
buffer
sent ACK
Received message:zoom_in
p 0.30859375 0.30859375
Small_d 1.649051855847802 1.2367888918858514
(-6.342091076785976, 5.198461630277608, 1.649051855847802, 1.2367888918858514)
buffer
sent ACK
Received message:zoom_in
p 0.3125 0.3125
Small_d 1.615397736340704 1.211548302255528
(-6.325264017032427, 5.185841335462446, 1.615397736340704, 1.211548302255528)
buffer
sent ACK
Received message:zoom_in
p 0.31640625 0.31640625
Small_d 

buffer
sent ACK
Received message:zoom_in
p 0.3046875 0.3046875
Small_d 0.9465221111371315 0.7098915833528485
(-5.9908262044306415, 4.935012976011107, 0.9465221111371315, 0.7098915833528485)
buffer
sent ACK
Received message:zoom_in
p 0.30859375 0.30859375
Small_d 0.9275916689143888 0.6956937516857916
(-5.98136098331927, 4.927914060177578, 0.9275916689143888, 0.6956937516857916)
buffer
sent ACK
Received message:zoom_in
p 0.3125 0.3125
Small_d 0.9086612266916462 0.6814959200187346
(-5.9718957622078985, 4.92081514434405, 0.9086612266916462, 0.6814959200187346)
buffer
sent ACK
Received message:zoom_in
p 0.31640625 0.31640625
Small_d 0.8897307844689035 0.6672980883516777
(-5.962430541096527, 4.913716228510522, 0.8897307844689035, 0.6672980883516777)
buffer
sent ACK
Received message:zoom_in
p 0.3203125 0.3203125
Small_d 0.8708003422461609 0.6531002566846207
(-5.9529653199851555, 4.9066173126769925, 0.8708003422461609, 0.6531002566846207)
buffer
sent ACK
Received message:zoom_in
p 0.32421875 0

ConnectionResetError: [Errno 104] Connection reset by peer

In [ ]:
import PIL.Image

image = PIL.Image.fromarray(frame)

image